```{contents}
```
## Data Encryption

**Data encryption in LangGraph** is a foundational security mechanism that ensures **confidentiality, integrity, and compliance** of workflow data — including **state, memory, tool outputs, user inputs, model responses, logs, and checkpoints** — throughout the lifecycle of graph execution.

LangGraph itself is **encryption-agnostic** (it provides integration points rather than a built-in cryptosystem), enabling production systems to enforce encryption using standard security infrastructure.

---

### **1. Why Encryption Matters in LangGraph Systems**

LangGraph workflows handle **high-risk data**:

* User conversations
* Enterprise documents
* API keys & secrets
* Personally identifiable information (PII)
* Model outputs with business logic

Without encryption, this data is vulnerable at **three layers**:

| Layer     | Threat                           |
| --------- | -------------------------------- |
| Storage   | Data leaks, disk theft           |
| Transit   | Man-in-the-middle attacks        |
| Execution | Memory inspection, logging leaks |

---

### **2. Encryption Scope in LangGraph**

| Component             | What Must Be Encrypted           |
| --------------------- | -------------------------------- |
| State Store           | Full graph state, metadata       |
| Checkpoint Store      | Snapshots, replay data           |
| Memory Store          | Vector embeddings, conversations |
| Logs & Traces         | Execution records                |
| Tool I/O              | Requests and responses           |
| Secrets               | API keys, tokens                 |
| Inter-Service Traffic | Agent communication              |

---

### **3. Encryption Architecture

```
Client
   │ TLS
   ▼
API Gateway
   │ TLS
   ▼
LangGraph Runtime
   │ Encrypted Channels
   ▼
State / Memory / Checkpoint Stores (AES-256)
```

Encryption is enforced in **two domains**:

| Domain          | Mechanism |
| --------------- | --------- |
| Data in Transit | TLS 1.3   |
| Data at Rest    | AES-256   |

---

### **4. Data at Rest Encryption**

All persistent components must encrypt data before storage.

#### **State & Checkpoint Encryption Example**

```python
from cryptography.fernet import Fernet
import json

key = Fernet.generate_key()
cipher = Fernet(key)

def encrypt_state(state: dict) -> bytes:
    return cipher.encrypt(json.dumps(state).encode())

def decrypt_state(blob: bytes) -> dict:
    return json.loads(cipher.decrypt(blob))
```

This encryption layer is applied before writing to:

* Redis
* PostgreSQL
* Object storage
* Vector databases

---

### **5. Data in Transit Encryption**

All service communication must use TLS:

* Client ↔ API
* API ↔ LangGraph
* LangGraph ↔ Tools
* LangGraph ↔ Databases

Example (FastAPI):

```python
uvicorn app:app --ssl-keyfile key.pem --ssl-certfile cert.pem
```

---

### **6. Encryption of Secrets & Credentials**

Secrets must **never** be stored in LangGraph state.

Use:

* HashiCorp Vault
* AWS KMS / Secrets Manager
* Azure Key Vault

```python
import os

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
```

---

### **7. Role-Based Encryption & Access Control**

Sensitive fields inside state may require **field-level encryption**:

```python
state = {
    "user_id": encrypt("12345"),
    "message": encrypt("confidential text"),
}
```

Only authorized nodes or agents may decrypt.

---

### **8. Production Security Controls**

| Control               | Purpose             |
| --------------------- | ------------------- |
| Key rotation          | Limit blast radius  |
| Audit logging         | Detect misuse       |
| Tamper detection      | Data integrity      |
| HSM integration       | Hardware security   |
| Encrypted backups     | Disaster recovery   |
| Zero-trust networking | Internal protection |

---

### **9. Compliance Alignment**

Encryption enables compliance with:

* GDPR
* HIPAA
* SOC 2
* ISO 27001
* PCI-DSS

---

### **10. Common Deployment Pattern**

| Layer     | Technology |
| --------- | ---------- |
| Transport | TLS 1.3    |
| Storage   | AES-256    |
| Keys      | KMS / HSM  |
| Secrets   | Vault      |
| Access    | RBAC + IAM |
| Auditing  | SIEM       |

---

### **11. Summary**

> In LangGraph, encryption is the **foundation of trust** that makes it safe to operate LLM workflows in production environments handling sensitive data.

Encryption protects:

**State • Memory • Agents • Tools • Logs • Users • Business Logic**



### Demonstration

In [1]:
# === Secure LangGraph Demo: Encrypted State + Workflow ===

from langgraph.graph import StateGraph, END
from typing import TypedDict
from cryptography.fernet import Fernet
import json

# -------------------------------
# 1. Key Management (normally via KMS/Vault)
# -------------------------------
KEY = Fernet.generate_key()
cipher = Fernet(KEY)

def encrypt(data: dict) -> bytes:
    return cipher.encrypt(json.dumps(data).encode())

def decrypt(blob: bytes) -> dict:
    return json.loads(cipher.decrypt(blob))

# -------------------------------
# 2. Encrypted Storage Simulation
# -------------------------------
DATABASE = {}

def save_state(thread_id, state):
    DATABASE[thread_id] = encrypt(state)

def load_state(thread_id):
    return decrypt(DATABASE[thread_id])

# -------------------------------
# 3. LangGraph State Definition
# -------------------------------
class State(TypedDict):
    count: int

# -------------------------------
# 4. Secure Nodes
# -------------------------------
def increment(state: State):
    state["count"] += 1
    return state

def stop_if_done(state: State):
    return END if state["count"] >= 3 else "increment"

# -------------------------------
# 5. Build Cyclic Graph
# -------------------------------
builder = StateGraph(State)

builder.add_node("increment", increment)
builder.set_entry_point("increment")

builder.add_conditional_edges("increment", stop_if_done, {
    "increment": "increment",
    END: END
})

graph = builder.compile()

# -------------------------------
# 6. Encrypted Execution
# -------------------------------
thread_id = "session-1"

initial_state = {"count": 0}
save_state(thread_id, initial_state)

while True:
    state = load_state(thread_id)
    result = graph.invoke(state)
    save_state(thread_id, result)
    if result["count"] >= 3:
        break

# -------------------------------
# 7. Final Decrypted Output
# -------------------------------
final_state = load_state(thread_id)
print("Encrypted DB contents:", DATABASE[thread_id])
print("Decrypted final state:", final_state)


Encrypted DB contents: b'gAAAAABpU3s68nZYRruO-GJUGqIJn2MmjqbgssO0Km1d0blqPJ4ZVc9uVlLkqjwcvBa1cW5iPe4fPu4RkYICgYgbtvZ2PNssBg=='
Decrypted final state: {'count': 3}
